<a href="https://colab.research.google.com/github/AndrewCrossman/Graduate_Research/blob/main/BNL_Scripts/Topas_Fits_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Topas Fits Analysis

*Analyzes fitted data parameters from Topas.*

##Imports

In [1]:
from google.colab import drive  #allows downloading and accessing files on drive
from google.colab import files  #allows downloading and accessing files on drive
import shutil                   #allows uploading files to drive
import glob                     #useful for collecting and organizing files
import csv
import numpy as np              #math library
import pandas as pd             #data organization library
import scipy as sp              #math library
from scipy import optimize      #math fitting library
import matplotlib.pylab as plt  #allows for plotting
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


## Extract Data from Google Drive

In [ ]:
drive.mount('/content/drive/', force_remount=True)

In [2]:
directory_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/CeO2_PDF_600_1sec/integration'
#tth_files = glob.glob(directory_path + '/*tth.xy')

In [ ]:
data_file = "directory_path"+"/Fit_results.txt"
data = pd.read_csv(data_file, names=["FileName", "r_wp", "lp", "lp_error"])

## Analyze Data

In [ ]:
pd.plotting.scatter_matrix(data, alpha=.2, figsize=(12,18));